<a href="https://colab.research.google.com/github/vidaurridante3/pointnet/blob/main/Pointnet_outline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [ ]:
# Point Net Backbone
class PointNetBackbone(nn.Module):
    def __init__(self):

        super(PointNetBackbone, self).__init__()


        # transformer encoders 1
        self.encoderlayer1 = nn.TransformerEncoderLayer(
            d_model = 3,
            nhead = 1,
            dim_feedforward=2048,
            dropout=0.2,
            activation= 'relu',
            layer_norm_eps=1e-05,
            batch_first=True,
            norm_first=False,
            bias=True,
            device=None,
            dtype=None
        )

        self.transformerencoder1 = nn.TransformerEncoder(
            self.encoderlayer1,
            num_layers = 4
            )


         # transformer encoders 2
        self.encoderlayer2 = nn.TransformerEncoderLayer(
            d_model = 64,
            nhead = 1,
            dim_feedforward=2048,
            dropout=0.2,
            activation= 'relu',
            layer_norm_eps=1e-05,
            batch_first=True,
            norm_first=False,
            bias=True,
            device=None,
            dtype=None
        )

        self.transformerencoder2 = nn.TransformerEncoder(
            self.encoderlayer2,
            num_layers = 4
            )

        self.pos_emb = nn.Embedding(
            num_embeddings=361,
            embedding_dim=64
            )

        # shared MLP 1
        self.mlp1 = nn.Sequential(
            nn.Conv1d(in_channels=3, out_channels=64, kernel_size=1),
            nn.ReLU(inplace = True),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=1),
            nn.ReLU(inplace = True),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=1),
            nn.ReLU(inplace = True)
        )


        # shared MLP 2

        self.mlp2 = nn.Sequential(
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=1),
            nn.ReLU(inplace = True),
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=1),
            nn.ReLU(inplace = True),
            nn.Conv1d(in_channels=128, out_channels=1024, kernel_size=1),
            nn.ReLU(inplace = True)
        )

        # "max pool" to get the global features
        self.final_conv = nn.Conv1d(in_channels=361,
            out_channels=1,
            kernel_size=1,
            stride=1,
            padding=0
        )


    def forward(self, x: NestedTensor):

        # get nested vector size
        data = x.tensors
        mask = x.mask

        # pass through first encoder layer
        output = self.transformerencoder1(data, mask)

        # pass through first shared MLP
        #into batch_size, num_features, num_points
        output = output.permute(0, 2, 1)
        output = self.mlp1(output)
        #batch_size, num_points, num_features
        output = output.permute(0, 2, 1)

        # pass through second encoder layer
        pos_emb = self.pos_emb
        output_with_pos = output + pos_emb
        output = self.transformerencoder2(output_with_pos, mask)

        # pass through second MLP
        #into batch_size, num_features, num_points
        output = output.permute(0, 2, 1)
        output = self.mlp2(output)
        #batch_size, num_points, num_features
        output = output.permute(0, 2, 1)


        #pass through max pool
        output = self.final_conv(output)

        return output

